# CUSIP / CIK Sample Mapping 


In [3]:
%%capture
%pip install neo4j python-dotenv pandas

In [4]:
import pandas as pd
from dotenv import load_dotenv
import os

pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 50)


## Get CUSIP/CIK Map

In [16]:
map_df = pd.read_csv('https://raw.githubusercontent.com/leoliu0/cik-cusip-mapping/master/cik-cusip-maps.csv')
map_df.cik = map_df.cik.astype(int)
map_df.sort_values(by='cusip6')

,cik,cusip6,cusip8
11622,1623590,000001,000001
22085,1453593,000001,000001
52449,1083672,000001,000001
21868,1076966,000001,000001
30876,1345016,000001,000001
...,...,...,...
39963,1099205,Z00107,Z0010713
26663,1674447,Z058Z0,Z058Z0
15271,880266,Z317FV,Z317FV
13757,1050446,Z317FW,Z317FW


## Get Some Sample SEC Graph Data

In [17]:
from neo4j import GraphDatabase

load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('OLD_NEO4J_URI')
NEO4J_AUTH = (os.getenv('OLD_NEO4J_USERNAME'), os.getenv('OLD_NEO4J_PASSWORD'))

with GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH) as driver:
    records, summary, keys = driver.execute_query('''MATCH(c:Company) RETURN c.cusip AS cusip, c.companyName AS name''', 
                         database='neo4j')
company_df = pd.DataFrame.from_records([x.data() for x in records])
company_df

,cusip,name
0,*W EXP 09/01/202,None
1,000000000,Asml Hldg Nv ADR
2,00000NANA,NOBLE CORP PLC
3,00000NRGV,Energy Vault
4,000053673,ASHTEAD GROUP PLC
...,...,...
16388,y8162g112,STEPPE CEMENT LTD
16389,y8564w103,Teekay Inc
16390,y8565n102,Teekay Tankers
16391,y8565n300,TEEKAY TANKERS LTD


## Format CUSIP 6 and 8
Test Merging on Map

In [18]:
## Padding of 3 zeros is suspect - likely has a padded zero. This is inconsistent among form13 filers
def get_cusip6(cusip):
    if cusip.startswith('000'):
        return cusip.upper()[1:7]
    return cusip.upper().upper()[:6]

In [19]:
# company_df['cusip6'] = company_df.cusip.str.upper().str[:6]
company_df['cusip6'] = company_df.cusip.apply(get_cusip6)
# company_df['cusip6_off'] = company_df.cusip.str.upper().str[1:7] # account for some form13 filings which use an extra padded zero
company_df['cusip8'] = company_df.cusip.str.upper().str[:8]
company_df

,cusip,name,cusip6,cusip8
0,*W EXP 09/01/202,None,*W EXP,*W EXP 0
1,000000000,Asml Hldg Nv ADR,000000,00000000
2,00000NANA,NOBLE CORP PLC,0000NA,00000NAN
3,00000NRGV,Energy Vault,0000NR,00000NRG
4,000053673,ASHTEAD GROUP PLC,000536,00005367
...,...,...,...,...
16388,y8162g112,STEPPE CEMENT LTD,Y8162G,Y8162G11
16389,y8564w103,Teekay Inc,Y8564W,Y8564W10
16390,y8565n102,Teekay Tankers,Y8565N,Y8565N10
16391,y8565n300,TEEKAY TANKERS LTD,Y8565N,Y8565N30


In [20]:
#cusip 8 merge
company_df.merge(map_df, on='cusip8', how='inner').groupby(['cusip', 'name','cusip6_x', 'cusip6_y', 'cusip8']).agg({'cik':list})

,,,,,cik
cusip,name,cusip6_x,cusip6_y,cusip8,
000360206,"AAON, Inc.",003602,000360,00036020,"[824142, 105634]"
000361105,AAR CORP,003611,000361,00036110,[1750]
00081T108,ACCO BRANDS CORP,0081T1,00081T,00081T10,[712034]
00081t108,ACCO Brands Corp.,0081T1,00081T,00081T10,[712034]
000868109,ACNB CORP,008681,000868,00086810,[715579]
...,...,...,...,...,...
y62267102,Navios Maritime Partners,Y62267,Y62267,Y6226710,"[1415921, 732857]"
y7542c130,SCORPIO TANKERS INC,Y7542C,Y7542C,Y7542C13,[1483934]
y8564w103,Teekay Inc,Y8564W,Y8564W,Y8564W10,"[911971, 1705931]"


In [21]:
#cusip 6 merge
company_df.merge(map_df, on='cusip6', how='inner').groupby(['cusip', 'name','cusip6']).agg({'cik':list, 'cusip8_x': list, 'cusip8_y': list})

,,,cik,cusip8_x,cusip8_y
cusip,name,cusip6,,,
000105510,AFLAC INC,001055,[4977],[00010551],[00105510]
000282410,ABBOTT LABS,002824,[1800],[00028241],[00282410]
001055102,AFLAC INC,001055,[4977],[00105510],[00105510]
001055902,AFLAC INC,001055,[4977],[00105590],[00105510]
001055952,AFLAC INC,001055,[4977],[00105595],[00105510]
...,...,...,...,...,...
y7542c130,SCORPIO TANKERS INC,Y7542C,"[1483934, 1483934]","[Y7542C13, Y7542C13]","[Y7542C10, Y7542C13]"
y8564w103,Teekay Inc,Y8564W,"[911971, 1705931, 911971]","[Y8564W10, Y8564W10, Y8564W10]","[Y8564W10, Y8564W10, Y8564W01]"
y8565n102,Teekay Tankers,Y8565N,"[1419945, 1419945]","[Y8565N10, Y8565N10]","[Y8565N10, Y8565N30]"


# Create Merged and Aggregated Dataset
Aggregate to cusip 6: This is what we would use to pull 10k by CIK and map back to cusip

In [22]:
merged_df = company_df.drop(columns=['cusip8']).merge(map_df.drop(columns=['cusip8']), on='cusip6', how='inner')
merged_df

,cusip,name,cusip6,cik
0,000105510,AFLAC INC,001055,4977
1,000282410,ABBOTT LABS,002824,1800
2,001055102,AFLAC INC,001055,4977
3,001055902,AFLAC INC,001055,4977
4,001055952,AFLAC INC,001055,4977
...,...,...,...,...
30101,y8897y198,TOP SHIPS INC,Y8897Y,1296484
30102,y8897y198,TOP SHIPS INC,Y8897Y,1296484
30103,y8897y198,TOP SHIPS INC,Y8897Y,1296484
30104,y8897y198,TOP SHIPS INC,Y8897Y,1296484


In [23]:
def most_frequent(s):
    x = s.tolist()
    return max(set(x), key=x.count)

cusip_cik_df = merged_df.groupby(['cusip6']).agg({'cik':most_frequent, 'name':set, 'cusip':set}).reset_index()
cusip_cik_df

,cusip6,cik,name,cusip
0,001055,4977,{AFLAC INC},"{001055102, 001055952, 000105510, 001055902}"
1,001084,1100441,{AGCO CORP},"{001084952, 001084902, 001084102}"
2,001228,1514281,{AG MORTGAGE INVESTMENT TRUST},{001228501}
3,00123Q,1423689,"{AGNC INVT CORP, AGNC Investment Corp.}","{00123q104, 00123Q104, 00123Q954, 00123Q904}"
4,00130H,874761,{AES CORP},"{00130H905, 00130H955, 00130H105, 00130h105}"
...,...,...,...,...
4643,Y81669,1328919,{STEALTHGAS INC},{Y81669106}
4644,Y8564W,911971,"{TEEKAY CORPORATION, Teekay Inc}","{Y8564W103, y8564w103, Y8564W903, Y8564W953}"
4645,Y8565N,1419945,"{TEEKAY TANKERS LTD, Teekay Tankers, TEEKAY TANKERS LTD-CLASS A}","{Y8565N900, y8565n102, Y8565N300, y8565n300, Y8565N950}"
4646,Y8897Y,1296484,{TOP SHIPS INC},{y8897y198}


## Write Sample Mapping

In [24]:
cusip_cik_df.to_csv('cik-cusip-sample-map.csv', index=False)